<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 1: Intro to Pandas


# LECTURE

## Start by importing necessary packages

In [2]:
import pandas as pd

## Read and save 311 Service Requests dataset as a pandas dataframe

In [3]:
df = pd.read_csv('https://nyu.box.com/shared/static/x3zfnpsva4kwcqj6amfchheszdv269lq.zip')

## Today's goal

Learn which 311 complaints are most common and which agencies are responsible for handling them. But first, let's take a look at the data, then clean it up!

## Preview the data contents

In [4]:
df.head() # defaults to providing the first 5 if you don't specify a number

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,39888071,08/01/2018 12:00:10 AM,08/01/2018 01:52:46 AM,DHS,Operations Unit - Department of Homeless Services,Homeless Person Assistance,NaN,Other,10029,200 EAST 109 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.793339,-73.942942,"(40.79333937834769, -73.9429417746998)"
1,39889166,08/01/2018 12:00:26 AM,08/18/2018 10:46:43 AM,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,RESIDENTIAL BUILDING,10031,528 WEST 136 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.820124,-73.953071,"(40.82012422332215, -73.9530712339799)"
2,39882869,08/01/2018 12:00:54 AM,08/01/2018 12:49:55 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11216,761 LINCOLN PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670809,-73.951399,"(40.67080917938279, -73.9513990916184)"
3,39894246,08/01/2018 12:01:00 AM,08/02/2018 10:30:00 PM,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),NaN,10010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.740262,-73.990517,"(40.74026158873342, -73.99051651686905)"
4,39881329,08/01/2018 12:01:00 AM,08/05/2018 12:00:00 AM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,11413,121-28 198 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.688144,-73.750990,"(40.68814402968042, -73.75098958473612)"


In [5]:
df.tail(10) # last 10 records in the dataframe

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
2859190,43619254,08/24/2019 01:58:34 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11226,1509 NOSTRAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.649391,-73.949411,"(40.649390802790116, -73.94941102713294)"
2859191,43623130,08/24/2019 01:58:58 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11234,FLATLANDS AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.626210,-73.927659,"(40.62621042607075, -73.92765889759266)"
2859192,43618563,08/24/2019 01:59:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11413,146 ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.661635,-73.762651,"(40.66163483733922, -73.76265143072415)"
2859193,43624826,08/24/2019 01:59:38 AM,NaN,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,11218,3403 14 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.641957,-73.981313,"(40.64195655592916, -73.98131320909779)"
2859194,43626512,08/24/2019 01:59:40 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034,65 PAYSON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.867405,-73.927765,"(40.867405209755695, -73.9277653007795)"
2859195,43619262,08/24/2019 02:00:14 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10463,3308 BAILEY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.879701,-73.901057,"(40.87970084316435, -73.9010571445122)"
2859196,43622052,08/24/2019 02:00:20 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11237,265 STOCKHOLM STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.701783,-73.920555,"(40.70178323733244, -73.92055493919345)"
2859197,43625918,08/24/2019 02:00:27 AM,NaN,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,11233,560 RALPH AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.669992,-73.922489,"(40.66999185896368, -73.9224889475533)"
2859198,43622055,08/24/2019 02:00:54 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2859199,43623704,08/24/2019 02:00:56 AM,NaN,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10303,2806 RICHMOND TERRACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636794,-74.153979,"(40.63679359185655, -74.15397942810047)"


In [6]:
df.sample(5) # random sample of size determined by you

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
2364351,42916588,06/08/2019 11:43:20 PM,06/16/2019 09:01:45 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10453,1501 UNDERCLIFF AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.845750,-73.924135,"(40.845749925937085, -73.92413470328373)"
1250106,41415155,01/13/2019 06:04:18 PM,01/15/2019 02:00:32 PM,DOT,Department of Transportation,Street Condition,"Rough, Pitted or Cracked Roads",Street,11236,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.628593,-73.905738,"(40.62859349083009, -73.90573812654965)"
719708,40720414,11/01/2018 10:29:18 AM,11/01/2018 10:36:30 AM,DOT,Department of Transportation,Street Condition,Plate Condition - Shifted,Street,10016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.750282,-73.981124,"(40.750281513068074, -73.98112401931803)"
963648,41095523,12/03/2018 04:06:17 PM,12/05/2018 08:54:39 AM,DOF,Correspondence Unit,DOF Property - Request Copy,Copy of Notice of Property Value,Property Address,10460,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76520,39972470,08/10/2018 06:15:18 PM,08/10/2018 08:12:24 PM,NYPD,New York City Police Department,Noise - Vehicle,Engine Idling,Street/Sidewalk,11004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.746039,-73.718531,"(40.74603884737402, -73.71853073537066)"


## How many records are in the dataset?

### `size` method

In [7]:
# how many cells are there in the data table?
df.size

117227200

In [8]:
# what if I only care about how many rows there are?
# the columns in the dataframe are like a list. 
# you can use a column name as an index to get one column from the dataframe

# size does include of null (empty) values
df['Facility Type'].size

2859200

### `count()` method

You can also use the `count()` function, which gives the count of values per column. `count()` doesn't include null (empty) values.

In [9]:
df.count()

Unique Key                        2859200
Created Date                      2859200
Closed Date                       2722927
Agency                            2859200
Agency Name                       2859200
Complaint Type                    2859200
Descriptor                        2816522
Location Type                     2242215
Incident Zip                      2746912
Incident Address                  2483931
Street Name                       2483750
Cross Street 1                    1719330
Cross Street 2                    1712706
Intersection Street 1              613896
Intersection Street 2              611987
Address Type                      2579813
City                              2725223
Landmark                           185198
Facility Type                      769320
Status                            2859200
Due Date                           978779
Resolution Description            2615253
Resolution Action Updated Date    2795696
Community Board                   

To just get the count in the "unique key" column:

In [10]:
df['Unique Key'].count()

2859200

### `info()` method

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2859200 entries, 0 to 2859199
Data columns (total 41 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Unique Key                      int64  
 1   Created Date                    object 
 2   Closed Date                     object 
 3   Agency                          object 
 4   Agency Name                     object 
 5   Complaint Type                  object 
 6   Descriptor                      object 
 7   Location Type                   object 
 8   Incident Zip                    object 
 9   Incident Address                object 
 10  Street Name                     object 
 11  Cross Street 1                  object 
 12  Cross Street 2                  object 
 13  Intersection Street 1           object 
 14  Intersection Street 2           object 
 15  Address Type                    object 
 16  City                            object 
 17  Landmark                   

## What are the distinct sets of values in columns that seem most useful?

### `set()` function for getting a set of unique values

Let's look at the "status" column. What are the status options for these 311 complaints?

In [12]:
set(df['Status'])

{'Assigned',
 'Closed',
 'Email Sent',
 'In Progress',
 'Open',
 'Pending',
 'Started',
 'Unspecified'}

In [13]:
set(df['Open Data Channel Type'])


{'MOBILE', 'ONLINE', 'OTHER', 'PHONE', 'UNKNOWN'}

In [14]:
set(df['Agency'])

{'ACS',
 'COIB',
 'DCA',
 'DCAS',
 'DCP',
 'DEP',
 'DFTA',
 'DHS',
 'DOB',
 'DOE',
 'DOF',
 'DOHMH',
 'DOITT',
 'DOT',
 'DPR',
 'DSNY',
 'DVS',
 'EDC',
 'FDNY',
 'HPD',
 'HRA',
 'MOC',
 'NYCEM',
 'NYPD',
 'OMB',
 'TAT',
 'TAX',
 'TLC'}

In [15]:
set(df['Complaint Type'])

{'"-->\'-->`-->&...',
 '$(sleep 11)',
 '${3804*3137}',
 '%2e%2e%2f%2e%2e%2f%2e%2e%2f...',
 '%2e%2e%5c%2e%2e%5c%2e%2e%5c...',
 '%2fetc%2fpasswd',
 '%E5%98%8A%E5%98%8DX-Injecti...',
 '%c0%ae/%c0%ae/%c0%ae/%c0%ae...',
 '%c0%ae/%c0%ae/%c0%ae/WEB-IN...',
 '%c0%ae/%c0%ae/WEB-INF/web.xml',
 '%c0%ae/WEB-INF/web.xml',
 '%{(#dm=@ognl.OgnlContext@DE...',
 '%{4761*8506}',
 '() { :;}; /bin/sleep 0',
 '() { :;}; /bin/sleep 11',
 '() { _; } >_',
 '(select extractvalue(xmltyp...',
 "(select load_file('\\\\\\\\615h...",
 '*)(!(objectClass=*)',
 '*)(objectClass=*',
 '.../....///.../....///.../....',
 '.../...//.../...//.../...//...',
 '..././..././..././..././......',
 '.../.\\.../.\\.../.\\.../.\\......',
 '.../Misc. Comments',
 '...\\./...\\./...\\./...\\./......',
 '...\\.\\...\\.\\...\\.\\...\\.\\......',
 '../../../../../../../../../...',
 '../../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml;x=',
 '../../WEB-INF/web.xml',
 '../../WEB-INF/web.xml;x=',
 '../WEB-INF/w

FYI you can also use `df['column_name'].unique()` to get a list of unique values.

## In-class exercise

[Create a Google Colab notebook](https://colab.research.google.com/notebook#create=true) called `lefture_1_exercise`. In it,

1. Read in the 311 dataset and save it as `df`. Remember there is a package you need to import in order for this to work.
2. Answer these two questions about the data, showing your code that produced the result in two individual code cells: 
    1. What is the minimum value in the `Created Date` column?
    1. What is the maximum value in the `Created Date` column?
3. Create a text cell where you will briefly discuss the min and max results: does anything about the resulting min and max values surprise you? What do think causes this? 
    - Hint: Look at the year. If you use `.head()` and `.tail()` you can find the real min and max dates. We'll learn more about how to properly handle dates in a later lecture.

## Excluding bad records from the dataframe

First, let's refresh ourselves on what the invalid complaint types are, by getting the distinct list of all complaint types

In [16]:
# the set() function returns all unique values in a column
set(df['Complaint Type'])

{'"-->\'-->`-->&...',
 '$(sleep 11)',
 '${3804*3137}',
 '%2e%2e%2f%2e%2e%2f%2e%2e%2f...',
 '%2e%2e%5c%2e%2e%5c%2e%2e%5c...',
 '%2fetc%2fpasswd',
 '%E5%98%8A%E5%98%8DX-Injecti...',
 '%c0%ae/%c0%ae/%c0%ae/%c0%ae...',
 '%c0%ae/%c0%ae/%c0%ae/WEB-IN...',
 '%c0%ae/%c0%ae/WEB-INF/web.xml',
 '%c0%ae/WEB-INF/web.xml',
 '%{(#dm=@ognl.OgnlContext@DE...',
 '%{4761*8506}',
 '() { :;}; /bin/sleep 0',
 '() { :;}; /bin/sleep 11',
 '() { _; } >_',
 '(select extractvalue(xmltyp...',
 "(select load_file('\\\\\\\\615h...",
 '*)(!(objectClass=*)',
 '*)(objectClass=*',
 '.../....///.../....///.../....',
 '.../...//.../...//.../...//...',
 '..././..././..././..././......',
 '.../.\\.../.\\.../.\\.../.\\......',
 '.../Misc. Comments',
 '...\\./...\\./...\\./...\\./......',
 '...\\.\\...\\.\\...\\.\\...\\.\\......',
 '../../../../../../../../../...',
 '../../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml;x=',
 '../../WEB-INF/web.xml',
 '../../WEB-INF/web.xml;x=',
 '../WEB-INF/w

Let's see how frequently these invalid Complaint Type values appear in the data.

Use `.groupby().size()` to get the count of 311 requests per complaint type value. This is very similar to a pivot table in Excel.

In [17]:
# remember .size gives you the count of cells across all columns in the dataframe
df.size

117227200

In [18]:
# to just get the total count of records in the dataset, we should get the size of the 'Unique Key' column
df['Unique Key'].size

2859200

In [46]:
# let's make sure we see everything
pd.options.display.max_rows = None


# watch out! the .groupby().size() function doesn't work that same way. it gets the count of number of rows in each group

df.groupby('Complaint Type').size()

# let's break down what this code is doing:
# first you need to group the records in the dataset based on their 'Complaint Type' value
# then you can count the records that have been grouped together by their shared 'Complaint Type" value

Complaint Type
"-->'-->`-->&...                                  1
$(sleep 11)                                       1
${3804*3137}                                      1
%2e%2e%2f%2e%2e%2f%2e%2e%2f...                    1
%2e%2e%5c%2e%2e%5c%2e%2e%5c...                    1
%2fetc%2fpasswd                                   1
%E5%98%8A%E5%98%8DX-Injecti...                    1
%c0%ae/%c0%ae/%c0%ae/%c0%ae...                    1
%c0%ae/%c0%ae/%c0%ae/WEB-IN...                    1
%c0%ae/%c0%ae/WEB-INF/web.xml                     1
%c0%ae/WEB-INF/web.xml                            1
%{(#dm=@ognl.OgnlContext@DE...                    1
%{4761*8506}                                      1
() { :;}; /bin/sleep 0                            1
() { :;}; /bin/sleep 11                           1
() { _; } >_                                      1
(select extractvalue(xmltyp...                    1
(select load_file('\\\\615h...                    1
*)(!(objectClass=*)                              

It looks like most invalid complaint types only have a few records. Try excluding all complaint type categories with < 4 records, assuming that all complaint type categories with < 4 instances in the data are bad data entries.

Why 4? It's arbitrary. We're looking for trends in the data in this case don't care about low frequency entries.

Create a dataframe that captures the count of records per `Complaint Type` value.

In [47]:
counts = df.groupby('Complaint Type').size().reset_index(name='count')
counts
# .reset_index(name='count') allows us to name the new column that contains the count of rows

,Complaint Type,count
0,"""-->'-->`-->&...",1
1,$(sleep 11),1
2,${3804*3137},1
3,%2e%2e%2f%2e%2e%2f%2e%2e%2f...,1
4,%2e%2e%5c%2e%2e%5c%2e%2e%5c...,1
5,%2fetc%2fpasswd,1
6,%E5%98%8A%E5%98%8DX-Injecti...,1
7,%c0%ae/%c0%ae/%c0%ae/%c0%ae...,1
8,%c0%ae/%c0%ae/%c0%ae/WEB-IN...,1
9,%c0%ae/%c0%ae/WEB-INF/web.xml,1


You can also use `.count()` but [the output is a little different](https://stackoverflow.com/questions/33346591/what-is-the-difference-between-size-and-count-in-pandas).

Create a "series" that only lists the `Complaint Type` values that have record counts > 4. (Remember: A single column from a pandas dataframe is called a series. It's essentially a list containing all the values in the column.) 

In [48]:
valid_complaint_types = counts['Complaint Type'][counts['count'] > 4]
valid_complaint_types

45                                     APPLIANCE
46                             Abandoned Vehicle
48                               Advocate - Lien
49                              Advocate - Other
50                               Advocate - RPIE
52                 Advocate-Co-opCondo Abatement
53                Advocate-Commercial Exemptions
55                  Advocate-Personal Exemptions
57                 Advocate-Prop Refunds/Credits
58                       Advocate-Property Value
60                                   Air Quality
61                              Alzheimer's Care
62                                  Animal Abuse
63                   Animal Facility - No Permit
64                              Animal in a Park
65                                  Animal-Abuse
67                                      Asbestos
68                              BEST/Site Safety
69                    Beach/Pool/Sauna Complaint
70                      Benefit Card Replacement
71                  

In [49]:
# go back to only displaying some
pd.reset_option("display.max_rows")

Filter our `df` dataframe to only keep the rows where the `Complaint Type` value is in the `valid_complaint_types` series we created in the previous step. Save the result in a new dataframe.

In [37]:
df_cleaned = df[df['Complaint Type'].isin(valid_complaint_types)]

How can we make sure this worked? Let's check how many records there were originally in `df` vs how many are in `df_cleaned`.

Before:

In [38]:
df['Unique Key'].size

2859200

After:

In [39]:
df_cleaned['Unique Key'].size

2859011

We can also print the set of complaint_type values from our cleaned dataframe to make sure they look correct.

In [40]:
df_cleaned['Complaint Type'].sample(10)

2139256                Noise - Commercial
1636808                  Blocked Driveway
2672790                             Noise
330863                       Damaged Tree
1014364                    HEAT/HOT WATER
450993                               Lead
1492841    DOF Property - Reduction Issue
45492                        Water System
561958                           ELECTRIC
2102934                    HEAT/HOT WATER
Name: Complaint Type, dtype: object

Great, now those invalid records will be excluded from our analysis!

Another approach to excluding those invalid records would be to use ["regex" (regular expressions)](https://www.w3schools.com/python/python_regex.asp) to find records with weird characters.

## Done with clean up! Time for the actual analysis: 
### Which 311 complaints are most common and which agencies are responsible for handling them?

Which complaints are the most common?

In [50]:
df_cleaned.groupby('Complaint Type').size().nlargest(15).reset_index(name='count')

# .reset_index(name='count') isn't necessary but it's helpful to include because it allows us to name the new column that contains the count of rows

,Complaint Type,count
0,Noise - Residential,236350
1,HEAT/HOT WATER,222722
2,Illegal Parking,195159
3,Request Large Bulky Item Collection,177175
4,Blocked Driveway,145446
5,Street Condition,97178
6,Noise - Street/Sidewalk,95977
7,UNSANITARY CONDITION,85904
8,Street Light Condition,77755
9,Water System,74139


Which agencies are responsible for handling these complaint categories?

In [51]:
df_cleaned.groupby(['Agency', 'Complaint Type']).size().nlargest(15).reset_index(name='count')

,Agency,Complaint Type,count
0,NYPD,Noise - Residential,236350
1,HPD,HEAT/HOT WATER,222722
2,NYPD,Illegal Parking,195159
3,DSNY,Request Large Bulky Item Collection,177175
4,NYPD,Blocked Driveway,145446
5,DOT,Street Condition,97178
6,NYPD,Noise - Street/Sidewalk,95977
7,HPD,UNSANITARY CONDITION,85904
8,DOT,Street Light Condition,77755
9,DEP,Water System,74139


Which agencies receive the most total 311 requests?

In [52]:
df_cleaned.groupby('Agency').size().nlargest(15).reset_index(name='count')

,Agency,count
0,NYPD,850077
1,HPD,603043
2,DSNY,420165
3,DOT,298879
4,DEP,207280
5,DOB,149656
6,DPR,119970
7,DOHMH,72253
8,DOF,41436
9,TLC,35730


What is the most frequent request per agency?

In [54]:
# first create a dataframe that contains the count of complaints per agency per comaplaint typw
agency_counts = df_cleaned.groupby(['Agency', 'Complaint Type']).size().reset_index(name='count')

# use "drop_duplicates" function to keep the row with the highest value per agency
agency_counts.sort_values('Agency').drop_duplicates('Agency')

,Agency,Complaint Type,count
0,ACS,Forms,364
1,COIB,Forms,7
2,DCA,Consumer Complaint,16385
4,DCAS,Comments,68
6,DCP,Research Questions,22
18,DEP,Water System,74139
26,DFTA,Housing - Low Income Senior,4375
30,DHS,Homeless Person Assistance,20991
45,DOB,Investigations and Discipline (IAD),841
55,DOE,School Maintenance,2372


## Saving our cleaned dataframe as a csv

In [45]:
df_cleaned.to_csv('cleaned_311_data.csv')

# HOMEWORK 1

[View](https://colab.research.google.com/github/afeld/python-public-policy/blob/master/hw_1.ipynb)